In [1]:
import json
from pprint import pformat
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px

In [2]:
nodes = {0: ["GPU0", "GPU1","GPU2","GPU3"],
           1: ["GPU0", "GPU1"],
           2: ["GPU0", "GPU1"]
        }

large_models = 5
small_models = 11
time_large = 6
time_small = 2

In [3]:
def load_worker_stats(worker_id, json_file):
    f = open(json_file)
    worker_dat = json.load(f)
    dat = []
    for epoch in worker_dat:
        for model in worker_dat[epoch]:
            dat.append([worker_id, epoch, model, worker_dat[epoch][model][0], worker_dat[epoch][model][1]])
    
    return dat

In [4]:
worker_jsons = {}
nworkers = 8
for i in range(nworkers):
    worker_jsons[i] = "stats/execution_time/worker_times" + str(i) + ".json"

In [30]:
all_dat_exec_times = []

In [31]:
for w in range(nworkers):
    all_dat_exec_times.extend(load_worker_stats(w, worker_jsons[w]))

In [32]:
df = pd.DataFrame(all_dat_exec_times, columns=['Worker', 'Epoch', 'Model','Start','End'])

In [33]:
df['Start'] = df['Start'].astype(int)
df['End'] = df['End'].astype(int)
df['Worker'] = df['Worker'].astype(str)
df['Model'] = df['Model'].astype(str)
df['Start'] = df['Start'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
df['End'] = df['End'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [35]:
df

,Worker,Epoch,Model,Start,End
0,0,epoch-0,model-group-3,2022-11-22 16:32:26,2022-11-22 16:32:46
1,0,epoch-0,model-group-7,2022-11-22 16:32:46,2022-11-22 16:32:48
2,0,epoch-0,model-group-8,2022-11-22 16:32:48,2022-11-22 16:32:50
3,0,epoch-0,model-group-6,2022-11-22 16:32:50,2022-11-22 16:32:52
4,0,epoch-0,model-group-9,2022-11-22 16:32:52,2022-11-22 16:32:54
...,...,...,...,...,...
65,7,epoch-0,model-group-1,2022-11-22 16:32:36,2022-11-22 16:32:46
66,7,epoch-0,model-group-3,2022-11-22 16:32:46,2022-11-22 16:32:56
67,7,epoch-0,model-group-2,2022-11-22 16:32:56,2022-11-22 16:33:06
68,7,epoch-0,model-group-4,2022-11-22 16:33:06,2022-11-22 16:33:21


In [40]:
fig = px.timeline(df, x_start="Start", x_end="End", y="Worker",color="Model")
fig.update_yaxes(autorange="reversed")
fig.show()

print("Node Configuration", pformat(nodes))
print("Number of large models", large_models)
print("Number of small models", small_models)
print("Time taken to train large model", time_large, "seconds")
print("Time taken to train small model", time_small, "seconds")

Node Configuration {0: ['GPU0', 'GPU1', 'GPU2', 'GPU3'], 1: ['GPU0', 'GPU1'], 2: ['GPU0', 'GPU1']}
Number of large models 11
Number of small models 5
Time taken to train large model 6 seconds
Time taken to train small model 2 seconds
